In [1]:
##使用python分词工具包jieba对微博文本进行分词
##分词原则：只保留名词和英文，过滤停止词和无意义词，过滤分词后单词个数小于3或大于70的微博，过滤低频词（出现次数小于3）
import jieba
import jieba.posseg as pseg
jieba.load_userdict('userdict.txt')
filter_list = ['网页','链接','正文']##过滤单词列表

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 2.084 seconds.
Prefix dict has been built succesfully.


In [2]:
##导入uid list
def load_uid():
    uid_list = []
    with open('../../data/uid.txt','r',encoding = 'utf-8') as f1:
        while True:
            line = f1.readline()
            if not line:
                break
            uid_list.append(line.strip())
        f1.close()
    return uid_list

In [3]:
##分词，只保留名词和英文，过滤停止词和filter
def split_text(text):
        result = pseg.cut(text) ##jieba分词
        word_list = []
        for w in result:
            #w.flag为词性
            flag = str(w.flag) 
            #w.word为单词
            word = str(w.word)
            word = word.lower()
            #print(w.word + '/' + w.flag)
            if word in filter_list: ##过滤单词列表
                continue
            if word in stopwords: ##过滤停止词
                continue
            if flag.startswith('n') or flag == 'eng': ##只保留名词和英文
                word_list.append(word)
        if len(word_list) < 3 or len(word_list) > 70: ##单词数小于3或大于70的微博过滤
            return []
        return word_list

In [4]:
##过滤低频词：出现次数小于三次
def low_fre_filter(result):
    fre = {}
    for item in result.values():
        for word in item:
            if word not in fre:
                fre[word] = 0
            fre[word] += 1
    tmp = {}
    avg = 0
    for id in result.keys():
        lis = result[id]
        tmp[id] = []
        for word in lis:
            if fre[word] > 2:
                tmp[id].append(word)
        avg += len(tmp[id])
    #print(avg/len(tmp))
    result = tmp
    return result

In [5]:
##读入停止词表
def load_stopwords():
    filename =  '../../data/text/stopwords.txt'
    stopwords = []
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = line.strip()
            stopwords.append(line)
        f.close()
    return stopwords

In [6]:
if __name__ == '__main__':
    uid_list = load_uid()
    for uid in uid_list:
        filename = '../../data/text/before_split/'+uid+'.txt'
        ## 读入微博文本
        with open(filename,'r',encoding="utf-8") as f:
            dic = f.readline().strip()
            dic = eval(dic)
            f.close()
        stopwords=load_stopwords() ##导入停止词
        result = {}
        for item in dic.keys():
            text = split_text(dic[item]) ##分词，返回分词后的list
            if len(text) == 0:
                continue
            result[item] = text
        retuslt = low_fre_filter(result) ##过滤低频词
        ##分词结果写入after_filter
        output = '../../data/text/after_filter/'+uid+'.dat'
        with open (output,'w',encoding='utf-8') as f2:
            for id in result.keys():
                f2.write(id+':')
                for item in result[id]:
                    f2.write(item+' ')
                f2.write('\n')
            f2.close()        